In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.api import VAR
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from pygam import GAM, s, LinearGAM

In [ ]:
df_train = pd.read_csv("../Daten/train_data.csv")

In [ ]:
df_train['Date and time'] = pd.to_datetime(df_train['Date and time'], format='mixed', dayfirst=True, errors='coerce')
df_train.set_index('Date and time', inplace=True)

In [ ]:
df_val = pd.read_csv("../Daten/validation_data.csv")

In [ ]:
df_val['Date and time'] = pd.to_datetime(df_val['Date and time'], format='mixed', dayfirst=True, errors='coerce')
df_val.set_index('Date and time', inplace=True)

# ARIMA

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


# ACF-Plot
plt.figure(figsize=(12, 5))
acf_plot = plot_acf(df_train["Power (kW)"], lags=50, title="Autokorrelationsfunktion (ACF)")
plt.xlabel("Verzögerungen")
plt.ylabel("Autokorrelation")

plt.ylim(-0.5, 1.1)
plt.grid(True)
plt.rcParams.update({'font.size': 16})
plt.tight_layout()
plt.show()

In [ ]:
# PACF-Plot
plt.figure(figsize=(12, 5))
pacf_plot = plot_pacf(df_train["Power (kW)"], lags=50, title="Partielle Autokorrelationsfunktion (PACF)")
plt.xlabel("Verzögerungen")
plt.ylabel("Partielle Autokorrelation")

# Lags und Zeitformat

plt.ylim(-0.5, 1.1)
plt.grid(True)
plt.rcParams.update({'font.size': 16})
plt.tight_layout()
plt.show()

In [ ]:
auto_arima_model = auto_arima(df_train["Power (kW)"].iloc[-144:], seasonal=False, trace=True, suppress_warnings=True, stepwise=False, m=1)
p, d, q = auto_arima_model.order

print(auto_arima_model.summary)

In [ ]:
print(auto_arima_model.summary)

In [ ]:
def check_stationarity(series):
    result = adfuller(series)
    print(f"p-Wert: {result[1]} (stationär, wenn < 0.05)")

check_stationarity(df_train["Power (kW)"].iloc[-144:])

In [ ]:
arima = ARIMA(df_train["Power (kW)"].iloc[-144:], order=auto_arima_model.order)
arima_fit = arima.fit()

In [ ]:
import pickle

with open('arima_model.pkl', 'wb') as file:
    pickle.dump(arima_fit, file)

In [ ]:
forecasts = arima_fit.get_forecast(steps=144)

In [ ]:
actual_values = df_val["Power (kW)"].iloc[:144]

mae = mean_absolute_error(actual_values, forecasts.predicted_mean)
rmse = root_mean_squared_error(actual_values, forecasts.predicted_mean)

print("Vorhersagen:", forecasts.predicted_mean)
print("Tatsächliche Werte:", actual_values.values)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
horizon = 144

plt.figure(figsize=(12, 6))
plt.title("ARIMA Vorhersage")
plt.plot(df_val.index[:horizon], df_val["Power (kW)"].iloc[:horizon], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:horizon], forecasts.predicted_mean[:horizon], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

## Ein Tages Vorhersage mit ARIMA

In [ ]:
sarima_modell = SARIMAX(df_train["Power (kW)"].iloc[-144:], order=(2,0,2), seasonal_order=(1,1,1,144))
fit = sarima_modell.fit()

In [ ]:
forecasts = fit.get_forecast(steps=144)

In [ ]:
forecasts.predicted_mean

In [ ]:
actual_values = df_val["Power (kW)"].iloc[:144]

mae = mean_absolute_error(actual_values, forecasts.predicted_mean)
rmse = root_mean_squared_error(actual_values, forecasts.predicted_mean)

print("Vorhersagen:", forecasts.predicted_mean)
print("Tatsächliche Werte:", actual_values.values)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("SARIMA Vorhersage")
plt.plot(df_val.index[:144], df_val["Power (kW)"].iloc[:144], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:144], forecasts.predicted_mean[:144], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

## SARIMAX Vorhersage mit exogenen Variablen

In [ ]:
df_exog = df_train[["Wind speed (m/s)"]]

In [ ]:
arima_model = SARIMAX(df_train["Power (kW)"], order=auto_arima_model.order, seasonal_order=(0,0,0,0), exog=df_train["Wind speed (m/s)"])
model_fit = arima_model.fit()

In [ ]:
import pickle

with open('sarimax_model.pkl', 'wb') as file:
    pickle.dump(model_fit, file)

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
model_fit.aic

In [ ]:
model_fit.plot_diagnostics(figsize=(15,12))

In [ ]:
df_train.describe()

In [ ]:
forecast = model_fit.get_forecast(steps=144, exog=df_val["Wind speed (m/s)"].iloc[:144])
forecast_ci = forecast.conf_int()

In [ ]:
forecast_ci

In [ ]:
actual_values = df_val["Power (kW)"].iloc[:144]

In [ ]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

mae = mean_absolute_error(actual_values, forecast.predicted_mean)
rmse = root_mean_squared_error(actual_values, forecast.predicted_mean)

print("Vorhersagen:", forecast.predicted_mean)
print("Tatsächliche Werte:", actual_values.values)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


In [ ]:
df_train

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("SARIMAX Vorhersagen (Mit Wind Daten)")
plt.plot(df_val.index[:144], actual_values[:144], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:144], forecast.predicted_mean[:144], label="Vorhersagen", linestyle="--")
plt.axvline(x=df_val.index[0], color="gray", linestyle="--", label="Trainings-/Validierungssplit")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

In [ ]:
pred = model_fit.get_prediction(start=pd.to_datetime("2019-06-30 23:00:00"), dynamic=False)

In [ ]:
pred.predicted_mean

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_train.index[-6:], df_train["Power (kW)"].iloc[-6:], label="Trainingsdaten")
plt.plot(df_train.index[-6:], pred.predicted_mean, label="Tatsächliche Werte (Validierung)")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(actual_values, forecast.predicted_mean)

print("Vorhersagen:", pred.predicted_mean)
print("Tatsächliche Werte:", df_train["Power (kW)"].iloc[-6:])
print("Root Mean Squared Error (RMSE):", rmse)


# GAM

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
data = df_train["Power (kW)"].iloc[:50000]
decomposition = seasonal_decompose(data, model="additive", period=144)
decomposition.plot()

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

data = df_train["Power (kW)"].iloc[-26200:]
decomposition = seasonal_decompose(data, model="additive", period=144)

fig, axes = plt.subplots(4, 1, figsize=(12, 6), sharex=True) 

axes[0].plot(decomposition.observed, label="Observed")
axes[0].set_title("Observed")
axes[0].legend()

axes[1].plot(decomposition.trend, label="Trend", color="orange")
axes[1].set_title("Trend")
axes[1].legend()

axes[2].plot(decomposition.seasonal, label="Seasonal", color="green")
axes[2].set_title("Seasonal")
axes[2].legend()

axes[3].plot(decomposition.resid, label="Residual", color="red")
axes[3].set_title("Residual")
axes[3].legend()

plt.tight_layout()
plt.show()

## Vorhersage mit Winddaten

In [ ]:
gam = GAM(s(0))

In [ ]:
train_size = 183000
forecast_size = 144

X_train = df_train["Wind speed (m/s)"].iloc[:train_size].values
X_test = df_val["Wind speed (m/s)"].iloc[:forecast_size].values

y_train = df_train["Power (kW)"].iloc[:train_size].values
y_test = df_val["Power (kW)"].iloc[:forecast_size].values


In [ ]:
gam.gridsearch(X_train, y_train)

In [ ]:
gam_fit = gam.fit(X_train, y_train)

In [ ]:
import pickle

with open('gam_model.pkl', 'wb') as file:
    pickle.dump(gam_fit, file)

In [ ]:
y_pred = gam.predict(X_test)

gam_mse = mean_squared_error(y_test, y_pred)
gam_mae = mean_absolute_error(y_test, y_pred)
gam_rmse = root_mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {gam_mse}')
print(f'Mean Absolute Error: {gam_mae}')
print(f'Root Mean Squared Error: {gam_rmse}')

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
error = []

i = 0
for y in y_pred:
    
    e = y_test[i] - y
    e = np.sqrt(e**2)
    error.append(e)
    i += 1


In [ ]:
print(error)

In [ ]:
errors

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_val.index[:forecast_size], y_test, label="Validierung")
plt.plot(df_val.index[:forecast_size], y_pred, label="Vorhersage")
plt.title("GAM Vorhersagen (Mit Wind Daten)")
plt.ylabel("Power (kW)")
plt.xlabel("Time")
plt.legend()
plt.show()

## Vorhersage ohne Winddaten

In [ ]:
gam = LinearGAM(s(0, lam=0.8))

In [ ]:
df_gam = pd.DataFrame()
df_gam['time'] = np.arange(len(df_train))

#df_gam['time_scaled'] = (df_gam['time'] - df_gam['time'].min()) / \
                            #(df_gam['time'].max() - df_gam['time'].min())

df_gam['month'] = df_train.index.month
df_gam['day_of_year'] = df_train.index.day_of_year
df_gam['hour'] = df_train.index.hour
df_gam['minute'] = df_train.index.minute

df_gam['hour_sin'] = np.sin(2 * np.pi * df_gam['hour'] / 24)
df_gam['hour_cos'] = np.cos(2 * np.pi * df_gam['hour'] / 24)

In [ ]:
df_gam_val = pd.DataFrame()
df_gam_val['time'] = np.arange(len(df_val))

#df_gam_val['time_scaled'] = (df_gam_val['time'] - df_gam_val['time'].min()) / \
                            #(df_gam_val['time'].max() - df_gam_val['time'].min())

df_gam_val['month'] = df_val.index.month
df_gam_val['day_of_year'] = df_val.index.day_of_year
df_gam_val['hour'] = df_val.index.hour
df_gam_val['minute'] = df_val.index.minute

df_gam_val['hour_sin'] = np.sin(2 * np.pi * df_gam_val['hour'] / 24)
df_gam_val['hour_cos'] = np.cos(2 * np.pi * df_gam_val['hour'] / 24)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

train_size = 144
forecast_size = 144

#X_train = df_gam.iloc[:train_size].values
#X_test = df_gam.iloc[train_size:train_size + forecast_size].values
X_train = df_gam[-train_size:]
X_val = df_gam_val[:forecast_size]

y_train = df_train["Power (kW)"].iloc[-train_size:].values

y_val = df_val["Power (kW)"].iloc[:forecast_size].values


In [ ]:
print("Shape von X:", X_train.shape)
print("Shape von y:", y_train.shape)

In [ ]:
result = gam.gridsearch(X_train, y_train).fit(X_train, y_train)

In [ ]:
result = gam.fit(X_train, y_train)

In [ ]:
result

In [ ]:
forecasts = result.predict(X_val)


In [ ]:
len(forecasts)

In [ ]:
actual_values = df_val["Power (kW)"].iloc[:forecast_size]

mae = mean_absolute_error(actual_values, forecasts)
rmse = root_mean_squared_error(actual_values, forecasts)

print("Vorhersagen:", forecasts)
print("Tatsächliche Werte:", actual_values.values)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("GAM Vorhersage")
plt.plot(df_val.index[:forecast_size], df_val["Power (kW)"].iloc[:forecast_size], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:forecast_size], forecasts[:forecast_size], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

# Prophet

In [ ]:
df_prophet = df_train.reset_index()[["Date and time", "Power (kW)"]]
df_prophet.columns = ["ds", "y"]
df_prophet_val = df_val.reset_index()[["Date and time", "Power (kW)"]]
df_prophet_val.columns = ["ds", "y"]

In [ ]:
df_prophet

In [ ]:
train_size = 183600
forecast_size = 144

In [ ]:
train_set = df_prophet.iloc[-train_size:]
val_set = df_prophet_val.iloc[:forecast_size]

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from hyperopt import fmin, tpe, hp, Trials

# Definiere das Ziel für die Optimierung
def objective(params):
    model = Prophet(
        changepoint_prior_scale=params['changepoint_prior_scale'],
        seasonality_prior_scale=params['seasonality_prior_scale'],
    )
    model.fit(train_set)
    df_cv = cross_validation(model, initial='365 days', period='180 days', horizon='365 days')
    df_p = performance_metrics(df_cv)
    return df_p['rmse'].mean()

# Suchraum definieren
space = {
    'changepoint_prior_scale': hp.loguniform('changepoint_prior_scale', -3, 0),
    'seasonality_prior_scale': hp.loguniform('seasonality_prior_scale', -3, 1),
}

# Optimierung durchführen
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)
print("Beste Parameter:", best)

In [ ]:
from prophet import Prophet

prophet = Prophet(
    changepoint_prior_scale=0.5,
    weekly_seasonality=True,
    daily_seasonality=True,
    yearly_seasonality=True
)

In [ ]:
prophet.fit(train_set)

In [ ]:
future = prophet.make_future_dataframe(periods=1, freq="10T")

In [ ]:
future

In [ ]:
forecast = prophet.predict(future)

In [ ]:
forecast

In [ ]:
actuals = val_set['y']
predictions = forecast['yhat'].iloc[-len(val_set):]

prophet_mse = mean_squared_error(actuals, predictions)
prophet_mae = mean_absolute_error(actuals, predictions)
prophet_rmse = root_mean_squared_error(actuals, predictions)

print(f'Mean Squared Error: {prophet_mse}')
print(f'Mean Absolute Error: {prophet_mae}')
print(f'Root Mean Squared Error: {prophet_rmse}')

In [ ]:
predictions

In [ ]:
plt.figure(figsize=(12, 6))
plt.title(label="Prophet Vorhersage")
plt.plot(val_set['ds'].iloc[:144], val_set['y'].iloc[:144], label="Validierungsdaten")
plt.plot(val_set['ds'].iloc[:144], predictions.iloc[:144], label="Vorhersage")
plt.ylabel("Power (kW)")
plt.xlabel("Time")
plt.legend()
plt.show()